In [24]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
from arcpy import env
from arcpy.sa import *
import math
from arcpy import metadata as md
import inspect
import datetime
import shutil
import csv

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

Created by Zac Canter for CBI 3/21

Required input data: Expanded WUI (to be created in another notebook), Roads and Gaps (the areas between parcels and SBC county shapefile)

**Overall process:**
- Select the parcels that intersect the expanded WUI
- Clip these parcels by the expanded WUI
- Subdivide those that are larger than 150 acres into 100 acre subdivisions
- Select parcels that are less than 10 acres, and dissolve boundaries
- Of those smaller dissolved boundaries, take those that are over 150 acres and subdivide them into 100 acre subdivisions
- Subdivide the roads and gap layers into 10 acre parcel subdivisions
- Merge the road and gap subdivisons, the 100 acre subdivisions, the 10 acre subdivisions, and all the remaining geometry
- Elimate geometry that is less than 1 acre
- Select parcels that are in the WUI Intermix/Interface areas and subdivide them into 10 acre subdivisions
- Add APN and create APN table

In [25]:
in_data = ["Expanded_WUI.shp", "Roads_Gaps.shp", "SBC_Boundary.shp", "SBC_Parcels.shp", "WUI_IntermixInterface.shp"]

Create the geodatabase

In [26]:
arcpy.ResetEnvironments()
gdb_name = "ReportingUnits.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
# arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

Copy all the starting data into the geodatabase and reproject it

In [27]:
for v in in_data:
    path = os.path.join(THIS_FOLDER, "in_data/{}".format(v))
    name = v.split(".")[0]
    arcpy.CopyFeatures_management(path, name)
    crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
    arcpy.Project_management(name, "{}_rpj".format(name), crs)
    arcpy.Delete_management(name)

Get the parcels that intersect with the WUI

In [28]:
selection = arcpy.SelectLayerByLocation_management("SBC_Parcels_rpj", "INTERSECT", "Expanded_WUI_rpj")
arcpy.CopyFeatures_management(selection, "WUI_Intersecting_Parcels")

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\ReportingUnits.gdb\\WUI_Intersecting_Parcels'>

Clip the WUI intersecting parcels by the expanded WUI

In [29]:
arcpy.Clip_analysis("WUI_Intersecting_Parcels", "Expanded_WUI_rpj", "WUI_Intersecting_Parcels_Clip")
arcpy.Clip_analysis("Roads_Gaps_rpj", "Expanded_WUI_rpj", "Roads_Gaps_Clip")

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\ReportingUnits.gdb\\Roads_Gaps_Clip'>

Create the variables for the cutoffs

In [30]:
acre = 43560

Select the parcels that are less than or equal to 150 acres and greater than or equal to 10 acres and add those to the reporting units list

In [39]:
LTE_Lg_GTE_Sm = arcpy.management.SelectLayerByAttribute("WUI_Intersecting_Parcels_Clip", "NEW_SELECTION", "Shape_Area <= {} And Shape_Area >= {}".format(150 * acre, 10 * acre), None)
arcpy.CopyFeatures_management(LTE_Lg_GTE_Sm, "LTE_Lg_GTE_Sm")
reporting_units = ["LTE_Lg_GTE_Sm"]

Select the parcels that are larger than 150 acres and subdivide them into 100 acre subdivisions, add those to the reporting units list

In [40]:
GT_Lg = arcpy.management.SelectLayerByAttribute("WUI_Intersecting_Parcels_Clip", "NEW_SELECTION", "Shape_Area > {}".format(150 * acre), None)
GT150_SubDiv100 = arcpy.management.SubdividePolygon(GT_Lg, "GT150_SubDiv100", "EQUAL_AREAS", None, "{} SquareFeet".format(100 * acre), None, 0, "STACKED_BLOCKS")
reporting_units.append("GT150_SubDiv100")

In [41]:
LT_Sm = arcpy.management.SelectLayerByAttribute(WUI_Intersecting_Parcels, "NEW_SELECTION", "Shape_Area < {}".format(10 * acre), None)
LT10_Dis = arcpy.gapro.DissolveBoundaries(LT_Sm, "LT10_Dis", "SINGLE_PART", None, None, None)

In [42]:
GT_Lg_Dis = arcpy.management.SelectLayerByAttribute(LT10_Dis, "NEW_SELECTION", "Shape_Area > {}".format(150 * acre), None)
GT_Lg_Dis_SubDiv = arcpy.management.SubdividePolygon(GT_Lg_Dis, "GT_Lg_Dis_SubDiv", "EQUAL_AREAS", None, "{} SquareFeet".format(100 * acre), None, 0, "STACKED_BLOCKS")
reporting_units.append("GT_Lg_Dis_SubDiv")

In [43]:
LTE_Lg = arcpy.management.SelectLayerByAttribute(LT10_Dis, "NEW_SELECTION", "Shape_Area <= {}".format(150 * acre), None)
arcpy.CopyFeatures_management(LTE_Lg, "LTE_Lg")
reporting_units.append("LTE_Lg")

In [44]:
arcpy.management.SubdividePolygon("Roads_Gaps_Clip", "Roads_Gaps_Clip_SubDiv", "EQUAL_AREAS", None, "{} SquareFeet".format(10 * acre), None, 0, "STACKED_BLOCKS")
reporting_units.append("Roads_Gaps_Clip_SubDiv")

In [45]:
arcpy.Merge_management(reporting_units, "Merge1")
arcpy.management.MultipartToSinglepart("Merge1", "Merge1_SP")

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\ReportingUnits.gdb\\Merge1_SP'>

In [46]:
selection = arcpy.management.SelectLayerByAttribute("Merge1_SP", "NEW_SELECTION", "Shape_Area < {}".format(10 * acre), None)
arcpy.management.Eliminate(selection, "Merge1_SP_Elim", "LENGTH", "", None)

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\ReportingUnits.gdb\\Merge1_SP_Elim'>

In [47]:
selection = arcpy.management.SelectLayerByAttribute("Merge1_SP_Elim", "NEW_SELECTION", "Shape_Area < {}".format(2 * acre), None)
arcpy.management.Eliminate(selection, "Merge1_SP_Elim_2", "LENGTH", "", None)

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\ReportingUnits.gdb\\Merge1_SP_Elim_2'>

In [69]:
selection = arcpy.SelectLayerByLocation_management("Merge1_SP_Elim_2", "INTERSECT", "WUI_IntermixInterface_rpj")
arcpy.CopyFeatures_management(selection, "Priority_Parcels")
arcpy.management.SubdividePolygon("Priority_Parcels", "Priority_Parcels_SubDiv", "EQUAL_AREAS", None, "{} SquareFeet".format(10 * acre), None, 0, "STACKED_BLOCKS")
reporting_units = ["Priority_Parcels_SubDiv"]

In [70]:
selection = arcpy.SelectLayerByLocation_management("Merge1_SP_Elim_2", "INTERSECT", "WUI_IntermixInterface_rpj", invert_spatial_relationship = "INVERT")
arcpy.CopyFeatures_management(selection, "Non_Priority_Parcels")
reporting_units.append("Non_Priority_Parcels")

In [71]:
arcpy.Merge_management(reporting_units, "Merge_Reporting_Units")
arcpy.management.MultipartToSinglepart("Merge_Reporting_Units", "Merge_Reporting_Units_SP")

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\ReportingUnits.gdb\\Merge_Reporting_Units_SP'>

In [72]:
fieldObs = arcpy.ListFields("Merge_Reporting_Units_SP")  

fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  

print(fieldNames)

['OBJECTID', 'Shape', 'apn', 'layer', 'situs1', 'situs2', 'acreage', 'landuse', 'usecode', 'tra', 'docnum', 'docdate', 'pcttransf', 'valreason', 'nontaxcode', 'sbeno', 'agpres', 'landvalue', 'strimpr', 'tradefix', 'livimpr', 'perpropdec', 'perspropun', 'mobilehome', 'exemptions', 'exempcode', 'homeowex', 'netsecval', 'net_impr', 'net_pers', 'net_unx', 'net_av', 'mailx1', 'mailx2', 'mnumber', 'mfrac', 'mdir', 'mstreet', 'mstrsuffix', 'munittype', 'munitnumb', 'pobox', 'mcity', 'mzip', 'mzipext', 'mstate', 'country', 'snum', 'sfra', 'sdir', 'sstreet', 'sstreetsuf', 'sunittype', 'sunitnumb', 'scity', 'szip', 'szipext', 'm_address1', 'm_address2', 'asmrollid', 'apn9', 'propid', 'sqfootage', 'yearbuilt', 'bedrooms', 'bathrooms', 'taxbill', 'permit', 'bkpglnk', 'gen_inquir', 'web_link', 'maptype', 'recmapnum', 'tractname', 'recmapbook', 'recmapdate', 'blocksecti', 'lotnum', 'unitnum', 'cbiinputs_', 'areaunit', 'recdatetim', 'recuserid', 'recuserlog', 'section', 'township', 'range', 'shape_le

In [73]:
FCfields = [f.name for f in arcpy.ListFields("Merge_Reporting_Units_SP")]
DontDeleteFields = ['Shape_Length', 'Shape_Area', 'OBJECTID', 'Shape']
fields2Delete = list(set(FCfields) - set(DontDeleteFields))
arcpy.DeleteField_management("Merge_Reporting_Units_SP", fields2Delete)

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\ReportingUnits.gdb\\Merge_Reporting_Units_SP'>

In [75]:
arcpy.analysis.SummarizeWithin("Merge_Reporting_Units_SP", "WUI_Intersecting_Parcels_Clip", "Reporting_Units", "KEEP_ALL", None, "ADD_SHAPE_SUM", "SQUAREKILOMETERS", "apn", "NO_MIN_MAJ", "NO_PERCENT", "apn_Summary")

id,value
0,C:\Users\zrcan\Google Drive\CBI\RPP\RPP_ReportingUnits\ReportingUnits.gdb\Reporting_Units
1,C:\Users\zrcan\Google Drive\CBI\RPP\RPP_ReportingUnits\ReportingUnits.gdb\apn_Summary


In [76]:
FCfields = [f.name for f in arcpy.ListFields("Reporting_Units")]
DontDeleteFields = ['Shape_Length', 'Shape_Area', 'OBJECTID', 'Shape', "Join_ID"]
fields2Delete = list(set(FCfields) - set(DontDeleteFields))
arcpy.DeleteField_management("Reporting_Units", fields2Delete)

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\ReportingUnits.gdb\\Reporting_Units'>

In [77]:
xcel_path = os.path.join(THIS_FOLDER, "APN_RU_Table.xlsx")
arcpy.conversion.TableToExcel("apn_Summary", xcel_path, "NAME", "CODE")

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\APN_RU_Table.xlsx'>

In [79]:
fieldObs = arcpy.ListFields("WUI_Intersecting_Parcels_Clip")  

fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  

print(fieldNames)

['OBJECTID', 'Shape', 'apn', 'layer', 'situs1', 'situs2', 'acreage', 'landuse', 'usecode', 'tra', 'docnum', 'docdate', 'pcttransf', 'valreason', 'nontaxcode', 'sbeno', 'agpres', 'landvalue', 'strimpr', 'tradefix', 'livimpr', 'perpropdec', 'perspropun', 'mobilehome', 'exemptions', 'exempcode', 'homeowex', 'netsecval', 'net_impr', 'net_pers', 'net_unx', 'net_av', 'mailx1', 'mailx2', 'mnumber', 'mfrac', 'mdir', 'mstreet', 'mstrsuffix', 'munittype', 'munitnumb', 'pobox', 'mcity', 'mzip', 'mzipext', 'mstate', 'country', 'snum', 'sfra', 'sdir', 'sstreet', 'sstreetsuf', 'sunittype', 'sunitnumb', 'scity', 'szip', 'szipext', 'm_address1', 'm_address2', 'asmrollid', 'apn9', 'propid', 'sqfootage', 'yearbuilt', 'bedrooms', 'bathrooms', 'taxbill', 'permit', 'bkpglnk', 'gen_inquir', 'web_link', 'maptype', 'recmapnum', 'tractname', 'recmapbook', 'recmapdate', 'blocksecti', 'lotnum', 'unitnum', 'cbiinputs_', 'areaunit', 'recdatetim', 'recuserid', 'recuserlog', 'section', 'township', 'range', 'shape_le

In [82]:
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable("Reporting_Units")
fieldmappings.addTable("WUI_Intersecting_Parcels_Clip")

keepers = ["Join_ID, apn"]

pcFieldIndex = fieldmappings.findFieldMapIndex("apn")
pcfieldmap = fieldmappings.getFieldMap(pcFieldIndex)
pcnew = pcfieldmap.outputField
pcnew.name = "apn_join"
pcnew.aliasName = "apn_join"
pcfieldmap.outputField = pcnew
pcfieldmap.mergeRule = "First"

fieldmappings.replaceFieldMap(pcFieldIndex, pcfieldmap)

for field in fieldmappings.fields:
    if field.name not in keepers:
        fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field.name))

arcpy.SpatialJoin_analysis("Reporting_Units", "WUI_Intersecting_Parcels_Clip", 
                           "Reporting_Units_APN", "JOIN_ONE_TO_ONE", "#", fieldmappings) 

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\ReportingUnits.gdb\\Reporting_Units_APN'>

In [83]:
FCfields = [f.name for f in arcpy.ListFields("Reporting_Units_APN")]
DontDeleteFields = ['Shape_Length', 'Shape_Area', 'OBJECTID', 'Shape', "Join_ID", "apn"]
fields2Delete = list(set(FCfields) - set(DontDeleteFields))
arcpy.DeleteField_management("Reporting_Units_APN", fields2Delete)

<Result 'C:\\Users\\zrcan\\Google Drive\\CBI\\RPP\\RPP_ReportingUnits\\ReportingUnits.gdb\\Reporting_Units_APN'>